# Data Exploration

In [1]:
import sqlite3
import pandas as pd

path = "../data/"  #Insert path here
database = path + "database.sqlite"
conn = sqlite3.connect(database)

cursor = conn.cursor()
print(cursor.fetchall())

matches_df = pd.read_sql(
    '''
    SELECT id,league_id,season,stage,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,LBH,LBD,LBA
    FROM Match m
    WHERE league_id == 1729
    AND season == '2015/2016'
    ORDER BY stage ASC
    ''', conn)

matches_df.head(5)

[]


,id,league_id,season,stage,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,LBH,LBD,LBA
0,4389,1729,2015/2016,1,1987032,9825,8654,0,2,1.29,5.5,12.0
1,4390,1729,2015/2016,1,1987033,8678,10252,0,1,2.05,3.3,4.0
2,4391,1729,2015/2016,1,1987034,8455,10003,2,2,1.40,4.5,10.0
3,4392,1729,2015/2016,1,1987035,8668,9817,2,2,1.75,3.8,5.0
4,4393,1729,2015/2016,1,1987036,8197,8472,4,2,2.00,3.4,4.2


In [2]:
#matches_df['home_goals']= matches_df.groupby('home_team_api_id')['home_team_goal'].cumsum().fillna(0)

In [3]:
#matches_df['away_goals']= matches_df.groupby('home_team_api_id')['away_team_goal'].cumsum().fillna(0)

In [4]:
#matches_df.head(3)

### Variables to consider

1. Home Team Total Wins	
2. Home Team Home wins	
3. Home Team Total Losses	
4. Home Team Home losses	
5. Away Team Rank	
6. Away Team Total Wins	
7. Away Team Away wins	
8. Away Team Total Losses	
9. Away Team Away Losses	
10. Home Team Total Goals	
11. Home Team Home Goals	
12. Home Team Total Goals allowed	
13. Home Team Home Goals Allowed	
14. Away Team Total Goals	
15. Away Team Away Goals	
16. Away Team Total Goals Allowed	
17. Away Team Away Goals Allowed	
18. Home Team Win y/n

### home / draw / away

In [5]:
matches_df['home']=0
matches_df['away']=0
matches_df['home_draw']=0
#matches_df['away_draw']=0

matches_df.loc[matches_df['home_team_goal'] > matches_df['away_team_goal'],'home']=1
matches_df.loc[matches_df['home_team_goal'] < matches_df['away_team_goal'],'away']=1
matches_df.loc[matches_df['home_team_goal'] == matches_df['away_team_goal'],'home_draw']=1
#matches_df.loc[matches_df['home_team_goal'] == matches_df['away_team_goal'],'away_draw']=1

In [6]:
matches_df.head(5)

,id,league_id,season,stage,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,LBH,LBD,LBA,home,away,home_draw
0,4389,1729,2015/2016,1,1987032,9825,8654,0,2,1.29,5.5,12.0,0,1,0
1,4390,1729,2015/2016,1,1987033,8678,10252,0,1,2.05,3.3,4.0,0,1,0
2,4391,1729,2015/2016,1,1987034,8455,10003,2,2,1.40,4.5,10.0,0,0,1
3,4392,1729,2015/2016,1,1987035,8668,9817,2,2,1.75,3.8,5.0,0,0,1
4,4393,1729,2015/2016,1,1987036,8197,8472,4,2,2.00,3.4,4.2,1,0,0


In [7]:
matches_df['lowest_odds']=0
matches_df['highest_odds']=0
matches_df['draw_only']=0
matches_df['home_only']=0
matches_df['away_only']=0

In [8]:
matches_df.head(5)

,id,league_id,season,stage,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,LBH,LBD,LBA,home,away,home_draw,lowest_odds,highest_odds,draw_only,home_only,away_only
0,4389,1729,2015/2016,1,1987032,9825,8654,0,2,1.29,5.5,12.0,0,1,0,0,0,0,0,0
1,4390,1729,2015/2016,1,1987033,8678,10252,0,1,2.05,3.3,4.0,0,1,0,0,0,0,0,0
2,4391,1729,2015/2016,1,1987034,8455,10003,2,2,1.40,4.5,10.0,0,0,1,0,0,0,0,0
3,4392,1729,2015/2016,1,1987035,8668,9817,2,2,1.75,3.8,5.0,0,0,1,0,0,0,0,0
4,4393,1729,2015/2016,1,1987036,8197,8472,4,2,2.00,3.4,4.2,1,0,0,0,0,0,0,0


In [9]:
matches_df['min_profit']=-10

### Lowest Odds

In [10]:
matches_df.loc[(matches_df[['LBH','LBD','LBA']].min(axis=1) == matches_df['LBH']) & (matches_df['home'] == 1), 'min_profit'] = matches_df['LBH']*10 
matches_df.loc[(matches_df[['LBH','LBD','LBA']].min(axis=1) == matches_df['LBA']) & (matches_df['away'] == 1), 'min_profit'] = matches_df['LBA']*10 
matches_df.loc[(matches_df[['LBH','LBD','LBA']].min(axis=1) == matches_df['LBD']) & (matches_df['home_draw'] == 1), 'min_profit'] = matches_df['LBD']*10 

In [11]:
matches_df['min_profit_cum']= matches_df['min_profit'].cumsum().fillna(0)

In [12]:
matches_df.head(10)

,id,league_id,season,stage,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,LBH,...,home,away,home_draw,lowest_odds,highest_odds,draw_only,home_only,away_only,min_profit,min_profit_cum
0,4389,1729,2015/2016,1,1987032,9825,8654,0,2,1.29,...,0,1,0,0,0,0,0,0,-10.0,-10.0
1,4390,1729,2015/2016,1,1987033,8678,10252,0,1,2.05,...,0,1,0,0,0,0,0,0,-10.0,-20.0
2,4391,1729,2015/2016,1,1987034,8455,10003,2,2,1.40,...,0,0,1,0,0,0,0,0,-10.0,-30.0
3,4392,1729,2015/2016,1,1987035,8668,9817,2,2,1.75,...,0,0,1,0,0,0,0,0,-10.0,-40.0
4,4393,1729,2015/2016,1,1987036,8197,8472,4,2,2.00,...,1,0,0,0,0,0,0,0,20.0,-20.0
5,4394,1729,2015/2016,1,1987037,10260,8586,1,0,1.67,...,1,0,0,0,0,0,0,0,16.7,-3.3
6,4395,1729,2015/2016,1,1987038,10261,8466,2,2,2.88,...,0,0,1,0,0,0,0,0,-10.0,-13.3
7,4396,1729,2015/2016,1,1987039,9850,9826,1,3,2.62,...,0,1,0,0,0,0,0,0,-10.0,-23.3
8,4397,1729,2015/2016,1,1987040,10194,8650,0,1,3.40,...,0,1,0,0,0,0,0,0,23.0,-0.3
9,4398,1729,2015/2016,1,1987041,8659,8456,0,3,5.50,...,0,1,0,0,0,0,0,0,16.7,16.4


### Highest Odds

In [13]:
matches_df['max_profit']=-10

In [14]:
matches_df.loc[(matches_df[['LBH','LBD','LBA']].max(axis=1) == matches_df['LBH']) & (matches_df['home'] == 1), 'max_profit'] = matches_df['LBH']*10
matches_df.loc[(matches_df[['LBH','LBD','LBA']].max(axis=1) == matches_df['LBA']) & (matches_df['away'] == 1), 'max_profit'] = matches_df['LBA']*10
matches_df.loc[(matches_df[['LBH','LBD','LBA']].max(axis=1) == matches_df['LBD']) & (matches_df['home_draw'] == 1), 'max_profit'] = matches_df['LBD']*10

In [15]:
matches_df['max_profit_cum']= matches_df['max_profit'].cumsum().fillna(0)

In [38]:
matches_df.head(200)

,id,league_id,season,stage,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,LBH,...,min_profit,min_profit_cum,max_profit,max_profit_cum,draw_profit,draw_profit_cum,home_profit,home_profit_cum,away_profit,away_profit_cum
0,4389,1729,2015/2016,1,1987032,9825,8654,0,2,1.29,...,-10.0,-10.0,120.0,120.0,-10.0,-10.0,-10.0,-10.0,120.0,120.0
1,4390,1729,2015/2016,1,1987033,8678,10252,0,1,2.05,...,-10.0,-20.0,40.0,160.0,-10.0,-20.0,-10.0,-20.0,40.0,160.0
2,4391,1729,2015/2016,1,1987034,8455,10003,2,2,1.40,...,-10.0,-30.0,-10.0,150.0,45.0,25.0,-10.0,-30.0,-10.0,150.0
3,4392,1729,2015/2016,1,1987035,8668,9817,2,2,1.75,...,-10.0,-40.0,-10.0,140.0,38.0,63.0,-10.0,-40.0,-10.0,140.0
4,4393,1729,2015/2016,1,1987036,8197,8472,4,2,2.00,...,20.0,-20.0,-10.0,130.0,-10.0,53.0,20.0,-20.0,-10.0,130.0
5,4394,1729,2015/2016,1,1987037,10260,8586,1,0,1.67,...,16.7,-3.3,-10.0,120.0,-10.0,43.0,16.7,-3.3,-10.0,120.0
6,4395,1729,2015/2016,1,1987038,10261,8466,2,2,2.88,...,-10.0,-13.3,33.0,153.0,33.0,76.0,-10.0,-13.3,-10.0,110.0
7,4396,1729,2015/2016,1,1987039,9850,9826,1,3,2.62,...,-10.0,-23.3,-10.0,143.0,-10.0,66.0,-10.0,-23.3,29.0,139.0
8,4397,1729,2015/2016,1,1987040,10194,8650,0,1,3.40,...,23.0,-0.3,-10.0,133.0,-10.0,56.0,-10.0,-33.3,23.0,162.0
9,4398,1729,2015/2016,1,1987041,8659,8456,0,3,5.50,...,16.7,16.4,-10.0,123.0,-10.0,46.0,-10.0,-43.3,16.7,178.7


### Draw / Home / Away Only

In [31]:
matches_df['draw_profit']=-10

In [32]:
matches_df.loc[(matches_df['home_draw'] == 1), 'draw_profit'] = matches_df['LBD']*10

In [35]:
matches_df['draw_profit_cum']= matches_df['draw_profit'].cumsum().fillna(0)

In [36]:
matches_df['home_profit']=-10

In [37]:
matches_df.loc[(matches_df['home'] == 1), 'home_profit'] = matches_df['LBH']*10

In [22]:
matches_df['home_profit_cum']= matches_df['home_profit'].cumsum().fillna(0)

In [23]:
matches_df['away_profit']=-10

In [24]:
matches_df.loc[(matches_df['away'] == 1), 'away_profit'] = matches_df['LBA']*10

In [25]:
matches_df['away_profit_cum']= matches_df['away_profit'].cumsum().fillna(0)

In [34]:
matches_df.tail(5)

,id,league_id,season,stage,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,LBH,...,min_profit,min_profit_cum,max_profit,max_profit_cum,draw_profit,draw_profit_cum,home_profit,home_profit_cum,away_profit,away_profit_cum
375,4704,1729,2015/2016,38,1987602,8466,9826,4,1,1.33,...,13.3,1464.7,-10.0,1693.6,-10.0,1021.0,-10,1151.6,-10.0,1392.9
376,4705,1729,2015/2016,38,1987603,10194,8654,2,1,3.40,...,-10.0,1454.7,-10.0,1683.6,-10.0,1011.0,-10,1185.6,-10.0,1382.9
377,4706,1729,2015/2016,38,1987604,10003,8456,1,1,6.50,...,-10.0,1444.7,-10.0,1673.6,43.3,1054.3,-10,1175.6,-10.0,1372.9
378,4707,1729,2015/2016,38,1987605,9817,8472,2,2,2.05,...,-10.0,1434.7,37.5,1711.1,37.5,1091.8,-10,1165.6,-10.0,1362.9
379,4708,1729,2015/2016,38,1987606,8659,8650,1,1,2.40,...,-10.0,1424.7,36.0,1747.1,36.0,1127.8,-10,1155.6,-10.0,1352.9


In [27]:
"""matches_df['home_wins_cum']=matches_df.groupby('home_team_api_id')['home'].cumsum()
matches_df['away_wins_cum']=matches_df.groupby('home_team_api_id')['away'].cumsum()
matches_df['home_draw_cum']=matches_df.groupby('home_team_api_id')['home_draw'].cumsum()
#matches_df['away_draw_cum']=matches_df.groupby('away_team_api_id')['away_draw'].cumsum()
"""

"matches_df['home_wins_cum']=matches_df.groupby('home_team_api_id')['home'].cumsum()\nmatches_df['away_wins_cum']=matches_df.groupby('home_team_api_id')['away'].cumsum()\nmatches_df['home_draw_cum']=matches_df.groupby('home_team_api_id')['home_draw'].cumsum()\n#matches_df['away_draw_cum']=matches_df.groupby('away_team_api_id')['away_draw'].cumsum()\n"

In [28]:
"""matches_df.groupby('stage').head(20)"""

"matches_df.groupby('stage').head(20)"